In [ ]:
%%capture --no-stderr
%pip install langchain langchain_community langchain_openai neo4j_graphrag langchain_neo4j neo4j

In [ ]:
from dotenv import load_dotenv

load_dotenv()

# 모델 생성

In [ ]:
import os
from langchain_openai import ChatOpenAI

#llm = ChatOpenAI(model="gpt-4o")
llm = ChatOpenAI(model=os.getenv("OPENAI_API_MODEL"))

# 문서를 변환하여 그래프를 생성한다.

In [ ]:
from langchain_core.documents import Document
from langchain_experimental.graph_transformers import LLMGraphTransformer

text = """
Marie Curie, born in 1867, was a Polish and naturalised-French physicist and chemist who conducted pioneering research on radioactivity.
She was the first woman to win a Nobel Prize, the first person to win a Nobel Prize twice, and the only person to win a Nobel Prize in two scientific fields.
Her husband, Pierre Curie, was a co-winner of her first Nobel Prize, making them the first-ever married couple to win the Nobel Prize and launching the Curie family legacy of five Nobel Prizes.
She was, in 1906, the first woman to become a professor at the University of Paris. 
"""

documents = [Document(page_content=text)]
llm_transformer = LLMGraphTransformer(llm=llm)
graph_documents = llm_transformer.convert_to_graph_documents(documents)

llm_transformer_filtered = LLMGraphTransformer(
    llm=llm,
    # allowed_nodes=["Person", "Country", "Organization"],
    # allowed_relationships=["NATIONALITY", "LOCATED_IN", "WORKED_AT", "SPOUSE"],
)
graph_documents_filtered = llm_transformer_filtered.convert_to_graph_documents(
    documents
)

# GraphRAG를 생성한다.

In [ ]:
from langchain_community.graphs import Neo4jGraph

neo4j_graph = Neo4jGraph(url="neo4j://localhost:7687", username="", password="")

# Add nodes to the graph
neo4j_graph.add_graph_documents(graph_documents_filtered)

# GraphRAG로부터 Retrieve를 포함하여 체인을 생성한다.

In [ ]:
from neo4j_graphrag.retrievers import Text2CypherRetriever
from langchain.chains import GraphCypherQAChain
from langchain_community.graphs import Neo4jGraph

# retriever = Text2CypherRetriever(
#     driver=GraphDatabase.driver("neo4j://localhost:7687", auth=None),
#     llm=llm,  # type: ignore
#     #neo4j_schema=neo4j_schema,
#     #examples=examples,
# )

neo4j_graph = Neo4jGraph(url="neo4j://localhost:7687", username="", password="")

chain = GraphCypherQAChain.from_llm(
    llm=llm, 
    graph=neo4j_graph, 
    verbose=True,  # verbose=True → Cypher 쿼리 출력
    #retriever=retriever,
    allow_dangerous_requests=True
)

# RAG를 이용하여 질문한다.

In [ ]:
question = """Who is Marie Curie?"""
result = chain.invoke(question)

print("답변:", result["result"], "\n")